In [1]:
from keras.callbacks import TensorBoard
%run data_gen.ipynb
%run models.ipynb

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    (None, 15, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 15, 28, 16)   160         root_word_input[0][0]            
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 7, 14, 16)    0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 7, 14, 8)     1160        max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

In [2]:
dg = DataGen()

n_features = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len


In [3]:
batch_size = 128
n_batches = int(len(dg.words) * .7 / batch_size) 
gen = dg.gen2(batch_size=batch_size, n_batches=n_batches)

In [4]:
# define model 
# train, infenc, infdec = seq2(n_features, n_features, 64, dg.word_feat_len)
train, infenc, infdec = conv_model(n_features, n_features, dg.word_feat_len, 64, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
tensorboard = TensorBoard(log_dir='logs')

In [8]:
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 2, callbacks=[tensorboard])

Epoch 1/2
5702/5702 [==============================] - 419s 73ms/step - loss: 0.0154 - acc: 0.9928
Epoch 2/2
  36/5702 [..............................] - ETA: 7:00 - loss: 0.0110 - acc: 0.9940

KeyboardInterrupt: 

In [9]:
test_n_batches, test_batch_size = 30, 100 
test_gen = dg.gen2(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

In [10]:
total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        X33 = X3[j].reshape((1, X3.shape[1])) 
        X11 = X1[j].reshape((1, X1.shape[1], X1.shape[2], 1))
        target = predict2(infenc, infdec, X11, X33, n_steps_out, n_features)
        root = ''.join(dg.one_hot_decode(X1[j]))#.replace('&', ' ')
        word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
        sims.append(dg.word_sim(word, targetS))
        if dg.one_hot_decode(y[j]) == dg.one_hot_decode(target):
            correct += 1
        if root.strip() in targetS.strip():
            in_word += 1
    print(b, root, word, targetS)
    total += test_batch_size
    
word_sim_average = sum(sims)/len(sims)
print('Word Similarity Average: {0:.2f}%'.format(word_sim_average))
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
print('Accuracy: %.2f%%' % (float(in_word)/float(total)*100.0))

0 matay           matayiyogaa&                    matayiyogaa&                   
1 sal             salissiyaro&                    salissiyaro&                   
2 maaret          maaretaro&                      maaretaro&                     
3 yedd            yeddaanaagoo&                   yeddaanaagoo&                  
4 cagg            cagissidori&                    cagissodi&                     
5 achch           achchissiyonaa&                 achchissiyonaa&                
6 homppat         homppatiyonii&                  homppatiyo&                    
7 paall           paallogeetoo&                   paallogeetoo&                  
8 baxx            baxissennee&                    baxissennee&                   
9 warqqall        warqqalliis&                    warqqalliis&                   
10 daadir          daadira&                        daadira&                       
11 paatt           paatissibeokkonaa&              paatissibeokkonaa&             
12 qoom       